# 📍 Problem Title: Students and Examinations

🔗 [LeetCode Problem 1280 – Students and Examinations](https://leetcode.com/problems/students-and-examinations/description/?envType=study-plan-v2&envId=30-days-of-pandas)

---

## 📝 Problem Description
We are given three tables:

**Students**

| Column       | Type |
|--------------|------|
| student_id   | int  |
| student_name | str  |

**Subjects**

| Column       | Type |
|--------------|------|
| subject_name | str  |

**Examinations**

| Column       | Type |
|--------------|------|
| student_id   | int  |
| subject_name | str  |

We need to return, for **every (student, subject) pair**, how many times that student attended exams for that subject.
- If the student never took the subject, the count should be `0`.
- The result must contain the following columns in order:
  - `student_id`
  - `student_name`
  - `subject_name`
  - `attended_exams`

---

## 🧾 Example

**Input:**

Students

| student_id | student_name |
|------------|--------------|
| 1          | Alice        |
| 2          | Bob          |

Subjects

| subject_name |
|--------------|
| Math         |
| Physics      |

Examinations

| student_id | subject_name |
|------------|--------------|
| 1          | Math         |
| 1          | Math         |
| 2          | Physics      |

**Output:**

| student_id | student_name | subject_name | attended_exams |
|------------|--------------|--------------|----------------|
| 1          | Alice        | Math         | 2              |
| 1          | Alice        | Physics      | 0              |
| 2          | Bob          | Math         | 0              |
| 2          | Bob          | Physics      | 1              |

---

## 🧠 Key Concepts
- **Cross join** → to generate all possible `(student, subject)` pairs.
- **Group by** → count how many times each pair appears in `Examinations`.
- **Left join + fillna** → attach counts to the full grid and set missing ones to `0`.
- **Sorting** → order result by `student_id`, then `subject_name`.

---

## 🐼 Pandas Outline
1. Perform a **cross join** between `students` and `subjects` → gives the full grid.
2. Group `examinations` by `(student_id, subject_name)` and count rows with `.size()`.
3. Left join the counts with the full grid.
4. Fill missing values with `0`, cast to `int`.
5. Sort and return columns in the required order.

---

✅ This solution ensures that even subjects with **no exams** for a student are included with `0`.


In [22]:
import pandas as pd

data = [[1, 'Alice'], [2, 'Bob'], [13, 'John'], [6, 'Alex']]
students = pd.DataFrame(data, columns=['student_id', 'student_name']).astype({'student_id':'Int64', 'student_name':'object'})
data = [['Math'], ['Physics'], ['Programming']]
subjects = pd.DataFrame(data, columns=['subject_name']).astype({'subject_name':'object'})
data = [[1, 'Math'], [1, 'Physics'], [1, 'Programming'], [2, 'Programming'], [1, 'Physics'], [1, 'Math'], [13, 'Math'], [13, 'Programming'], [13, 'Physics'], [2, 'Math'], [1, 'Math']]
examinations = pd.DataFrame(data, columns=['student_id', 'subject_name']).astype({'student_id':'Int64', 'subject_name':'object'})

In [23]:
students

,student_id,student_name
0,1,Alice
1,2,Bob
2,13,John
3,6,Alex


In [24]:
subjects

,subject_name
0,Math
1,Physics
2,Programming


In [25]:
examinations

,student_id,subject_name
0,1,Math
1,1,Physics
2,1,Programming
3,2,Programming
4,1,Physics
5,1,Math
6,13,Math
7,13,Programming
8,13,Physics
9,2,Math


In [67]:
def students_and_examinations(students : pd.DataFrame,
                              subjects : pd.DataFrame,
                              examinations : pd.DataFrame) -> pd.DataFrame:

    # 1-) Cartesian product : every (student, subject) pair.
    all_pairs = (
        students.assign(_k = 1)
        .merge(subjects.assign(_k = 1) , on = "_k")
        .drop(columns = "_k")
    )
    # 2-) Count exams per (student_id , subject_name
    counts = (
        examinations
        .groupby(["student_id", "subject_name"])
        .size()
        .reset_index(name = "attended_exams")
    )

    # 3-) Left-join counts onto all price; fill missing with 0
    out = (
        all_pairs
        .merge(counts, on=["student_id", "subject_name"], how="left")
        .fillna({"attended_exams": 0})
    )
    out["attended_exams"] = out["attended_exams"].astype(int)

    return out




In [68]:
students_and_examinations(students,subjects,examinations)

,student_id,student_name,subject_name,attended_exams
0,1,Alice,Math,3
1,1,Alice,Physics,2
2,1,Alice,Programming,1
3,2,Bob,Math,1
4,2,Bob,Physics,0
5,2,Bob,Programming,1
6,13,John,Math,1
7,13,John,Physics,1
8,13,John,Programming,1
9,6,Alex,Math,0
